# Large Landslides of North America

## Run Google earth Engine (GEE)

In [1]:
import ee
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AbUR2VNwSijxNQpXuUVU7sSfMiLpFEaKay8I5fFfRtYzU7yR9v6JBOIZMA0

Successfully saved authorization token.


## Data source

1. [Copernicus_S1_GRD](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S1_GRD) by European Space Agency(ESA)
2. USGS NAIP Imagery NDVI (GEE basemap layer)
3. USGS NAIP Imagery False Color
4. Verified landslides of North America collected by Earth Lab at the University of Colorado Boulder

## Import libraries and packages

In [2]:
# Required libraries and packages
import os
import json
import earthpy as et
import pandas as pd
import datetime
import pathlib
import folium
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import ee
import geemap

import shapely.geometry as sgeo
import IPython.display as disp
import geemap.foliumap as geemap

from shapely.geometry import Point
#from src.det import det
#from src.create_geodataframe import create_geodataframe
from scipy.stats import norm, gamma, f, chi2
%matplotlib inline

In [3]:
def chi2cdf(chi2, df):
    """Calculates Chi square cumulative distribution function for
       df degrees of freedom using the built-in incomplete gamma
       function gammainc().
    """
    return ee.Image(chi2.divide(2)).gammainc(ee.Number(df).divide(2))

def det(im):
    """Calculates determinant of 2x2 diagonal covariance matrix."""
    return im.expression('b(0)*b(1)')

## Set working directory

In [4]:
# Change directory to landslide-detect data path
data_path = os.path.join(et.io.HOME, "earth-analytics", "landslide-detect")
if os.path.exists(data_path):
    os.chdir(data_path)
else:
    os.makedirs(data_path)
    print('The new directory is created!')
    os.chdir(data_path)

data_path

'C:\\Users\\nasim\\earth-analytics\\landslide-detect'

In [5]:
%%bash
find .

.
./landslides.verified.csv


## Create dataframe from csv file

In [6]:
# Create DataFrame and open landslide file

landslide_gdf = gpd.read_file('landslides.verified.csv')
landslide_gdf.head(2)

,slide.id,slide.date,location,type,trigger,size,lon,lat,location_accuracy,event_title,admin_division_name,ge.lat,ge.lon,is.exact,slide.index,geometry
0,8321,2015-09-21T00:00:00Z,"Birken, BC, Canada",landslide,downpour,medium,-122.6205,50.479,5km,"Birken, BC, Canada",British Columbia,NA,NA,FALSE,1,None
1,7757,2015-12-07T18:00:00Z,Edmonds,mudslide,rain,medium,-122.3805278,47.70679444,5km,Edmonds,Washington,47.70679444,-122.3805278,TRUE,2,None


In [7]:
# Extract verified landslides of Highway 145, Colorado
Highway145_df = landslide_gdf[landslide_gdf['location'].str.contains
                              ('Highway 145')]
Highway145_df

,slide.id,slide.date,location,type,trigger,size,lon,lat,location_accuracy,event_title,admin_division_name,ge.lat,ge.lon,is.exact,slide.index,geometry
48,9114,2016-07-20T14:30:00Z,"Highway 145 between Sawpit and Placerville, Sa...",mudslide,downpour,medium,-108.0216,37.9939,5km,"Highway 145 between Sawpit and Placerville, Sa...",Colorado,NA,NA,FALSE,62,None
57,9112,2016-07-20T14:30:00Z,"Highway 145 between Sawpit and Placerville, Sa...",mudslide,downpour,medium,-108.0336,37.9979,5km,"Highway 145 between Sawpit and Placerville, Sa...",Colorado,NA,NA,FALSE,72,None
60,9113,2016-07-20T14:30:00Z,"Highway 145 between Sawpit and Placerville, Sa...",mudslide,downpour,medium,-108.0294,37.9948,5km,"Highway 145 between Sawpit and Placerville, Sa...",Colorado,NA,NA,FALSE,75,None
63,9116,2016-07-20T14:30:00Z,"Highway 145 between Sawpit and Placerville, Sa...",mudslide,downpour,medium,-108.012,37.9933,5km,"Highway 145 between Sawpit and Placerville, Sa...",Colorado,NA,NA,FALSE,78,None
67,9110,2016-07-20T14:30:00Z,"Highway 145 between Sawpit and Placerville, Sa...",mudslide,downpour,medium,-108.044,38.0072,5km,"Highway 145 between Sawpit and Placerville, Sa...",Colorado,NA,NA,FALSE,82,None
70,9117,2016-07-20T14:30:00Z,"Highway 145 between Sawpit and Placerville, Sa...",mudslide,downpour,medium,-108.0364,38.0008,5km,"Highway 145 between Sawpit and Placerville, Sa...",Colorado,NA,NA,FALSE,85,None
71,9109,2016-07-20T14:30:00Z,"Highway 145 between Sawpit and Placerville, Sa...",mudslide,downpour,medium,-108.0462,38.009,5km,"Highway 145 between Sawpit and Placerville, Sa...",Colorado,NA,NA,FALSE,86,None
84,9107,2016-07-20T14:30:00Z,"Highway 145 between Sawpit and Placerville, Sa...",mudslide,downpour,medium,-108.0511,38.0131,5km,"Highway 145 between Sawpit and Placerville, Sa...",Colorado,NA,NA,FALSE,100,None
96,9108,2016-07-20T14:30:00Z,"Highway 145 between Sawpit and Placerville, Sa...",mudslide,downpour,medium,-108.0488,38.0115,5km,"Highway 145 between Sawpit and Placerville, Sa...",Colorado,NA,NA,FALSE,114,None
105,9115,2016-07-20T14:30:00Z,"Highway 145 between Sawpit and Placerville, Sa...",mudslide,downpour,medium,-108.0164,37.9926,5km,"Highway 145 between Sawpit and Placerville, Sa...",Colorado,NA,NA,FALSE,123,None


In [8]:
# Extract verified large landslides
large_ls = landslide_gdf[landslide_gdf['size'].str.contains
                              ('large')]
large_ls.describe()

,slide.id,slide.date,location,type,trigger,size,lon,lat,location_accuracy,event_title,admin_division_name,ge.lat,ge.lon,is.exact,slide.index,geometry
count,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,0
unique,20,18,20,4,4,2,20,20,3,20,10,5,5,2,20,0
top,8728,2015-10-16T00:00:00Z,Broadmoor Bluffs neighborhood,landslide,rain,large,-104.8318,38.7612,1km,Broadmoor Bluffs neighborhood,NA,NA,NA,FALSE,15,NaN
freq,1,3,1,9,8,18,1,1,7,1,8,16,16,16,1,NaN


In [9]:
# Display all verified Highway 145 landslides
large_ls_map = folium.Map(
    location=[38.0021, -108.0372],
    zoom_start=4,
    width=1000,
    height=600,
    tiles='Stamen terrain')


for index, row in large_ls.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=row['slide.id'],
        icon=folium.Icon(color="red")
    ).add_to(large_ls_map)

large_ls_map

## Create interactive map of landslides of AOI (Highway 145)

In [10]:
# Display all verified Highway 145 landslides
landslide_map = folium.Map(
    location=[38.0021, -108.0372],
    zoom_start=14,
    width=1000,
    height=600,
    tiles='Stamen terrain')


for index, row in Highway145_df.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=row['slide.id'],
        icon=folium.Icon(color="red")
    ).add_to(landslide_map)

landslide_map

## AOI (Area of Interest)

In [11]:
# Make split view map half Satellite and half NAIP Imagery False Color

split_roi = geemap.Map(location=[38.0021, -108.0372], zoom_start=14)
split_roi.add_basemap('SATELLITE')
split_roi.add_basemap('USGS NAIP Imagery False Color')
# Map.add_basemap('USGS NAIP Imagery NDVI')

split_roi.split_map(left_layer='SATELLITE',
                    right_layer='USGS NAIP Imagery False Color')

split_roi

## Data avalability

In [12]:
# Sentinel-1 images availability of Highway 145 landslide area
event_date = ee.Date('2016-07-20')

# Select one year surrounding the event
start_date = event_date.advance(-180, 'days')
end_date = event_date.advance(180, 'days')

'Sentinel-1 image data range is between {} and {}'.format(
    start_date.format('YYYY-MM-dd').getInfo(),
    end_date.format('YYYY-MM-dd').getInfo())

'Sentinel-1 image data range is between 2016-01-22 and 2017-01-16'

In [13]:
## define the center coordinates of region of interest
center_point = ee.Geometry.Point([-108.0372, 38.0021])

# Create an Earth Engine Bbox using the center coordinates and dimensions
width = 1000
#height_degrees = 600
bbox = center_point.buffer(width / 2).bounds()

# Print the bounding box coordinates with four decimal places
print("Bounding box coordinates: ", [[round(x, 4) for x in coords] for coords
                                     in bbox.coordinates().getInfo()[0]])

bbox

Bounding box coordinates:  [[-108.0429, 37.9976], [-108.0315, 37.9976], [-108.0315, 38.0066], [-108.0429, 38.0066], [-108.0429, 37.9976]]


ee.Geometry({
  "functionInvocationValue": {
    "functionName": "Geometry.bounds",
    "arguments": {
      "geometry": {
        "functionInvocationValue": {
          "functionName": "Geometry.buffer",
          "arguments": {
            "distance": {
              "constantValue": 500.0
            },
            "geometry": {
              "functionInvocationValue": {
                "functionName": "GeometryConstructors.Point",
                "arguments": {
                  "coordinates": {
                    "constantValue": [
                      -108.0372,
                      38.0021
                    ]
                  }
                }
              }
            }
          }
        }
      }
    }
  }
})

## Collect SAR Images

In [14]:
# Collect and filter Sentinel-1 images by time and region of interest
start_date = '2016-01-22'
end_date = '2017-01-16'
sentinel_1 = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
              .filterBounds(bbox)
              .filterDate(start_date, end_date)
              .filter(ee.Filter.listContains('transmitterReceiverPolarisation'
                                             ,'VV'))
              .filter(ee.Filter.listContains('transmitterReceiverPolarisation'
                                             ,'VH'))
             )

image_collection = sentinel_1.filter(ee.Filter.eq('orbitProperties_pass', 
                                                  'ASCENDING'))

orbit_num = (image_collection.aggregate_array('relativeOrbitNumber_start')
             .getInfo())
if orbit_num:
    orbit_num = orbit_num[0]

look_angle = (image_collection.aggregate_array('orbitProperties_pass')
              .getInfo())
if look_angle and len(look_angle) > 0:
    look_angle = look_angle[0]

print('The Relative Orbit Number for ROI is: ', orbit_num)
print('The orbitology is: ', look_angle)

image_collection

The Relative Orbit Number for ROI is:  49
The orbitology is:  ASCENDING


In [19]:
# Retrieve all images in the collection
image_list = image_collection.toList(image_collection.size())

# Print date of each image in the list
print('All images in collection:')
for i in range(image_list.size().getInfo()):
    image = ee.Image(image_list.get(i))
    acquisition_time = ee.Date(image.get('system:time_start'))
    print('Date:', acquisition_time.format('YYYY-MM-dd').getInfo())

All images in collection:
Date: 2016-03-08
Date: 2016-08-23
Date: 2016-10-28
Date: 2016-11-21
Date: 2016-12-15
Date: 2017-01-08


In [23]:
# Retrieve acquisition date of each image in the collection as a list
timestamplist = (image_collection.aggregate_array('system:time_start')
                 .map(lambda t: ee.String('T').cat(ee.Date(t).format(
                     'YYYYMMdd')))
                 .getInfo())

timestamplist

['T20160308', 'T20160823', 'T20161028', 'T20161121', 'T20161215', 'T20170108']

## Convert and clip Image collection

In [27]:
im_list = image_collection.toList(image_collection.size())

# clip our list of images to the aoi geometry
def clip_img(img):
    """
    Clips a list of images to our aoi geometry.

    Returns
    -------
    list
        clipped images to aoi

    """
    return ee.Image(img).clip(bbox)

im_list = ee.List(im_list.map(clip_img))
im_list.get(0)
ee.Image(im_list.get(0)).bandNames().getInfo()
im_list.length().getInfo()

6

## Change detection

In [43]:
# Add EE drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

In [48]:
def selectvv(current):
    return ee.Image(current).select('VV')

vv_list = im_list.map(selectvv)

location = [-108.0372, 38.0021]

# Create a Folium map centered at the centroid location
mp = folium.Map(location=[38.0021, -108.0372], zoom_start=11)
rgb_images = (ee.Image.rgb(vv_list.get(0), vv_list.get(1), vv_list.get(2))
              .log10().multiply(10))
mp.add_ee_layer(rgb_images, {'min': -20,'max': 0}, 'rgb composite')
mp.add_child(folium.LayerControl())

In [ ]:
# Get the first and last images in the collection
first_image = image_collection.first()
last_image = image_collection.sort('system:time_start', False).first()

# Compute the difference image
diff_image = last_image.subtract(first_image)

# Define the visualization parameters for the difference image
diff_vis_params = {'min': -5, 'max': 5, 'palette': ['red', 'yellow', 'green']}

# Display the difference image on a map
Map.addLayer(diff_image, diff_vis_params, 'Change Map')


## Change map

In [55]:
def sample_vv_imgs(j):
    """Samples the test statistics Rj in the region bbox."""
    j = ee.Number(j)
    # Get the factors in the expression for Rj.
    sj = vv_list.get(j.subtract(1))
    jfact = j.pow(j).divide(j.subtract(1).pow(j.subtract(1)))
    sumj = ee.ImageCollection(vv_list.slice(0, j)).reduce(ee.Reducer.sum())
    sumjm1 = ee.ImageCollection(vv_list.slice(
        0, j.subtract(1))).reduce(ee.Reducer.sum())
    # Put them together.
    Rj = sumjm1.pow(j.subtract(1)).multiply(
        sj).multiply(jfact).divide(sumj.pow(j)).pow(5)
    # Sample Rj.
    sample = (Rj.sample(region=bbox, scale=10, numPixels=1000, seed=123)
              .aggregate_array('VV_sum'))
    return sample

In [56]:
# Sample the first few list indices.
samples = ee.List.sequence(2, 5).map(sample_vv_imgs)

# Calculate and display the correlation matrix.
np.set_printoptions(precision=2, suppress=True)

In [60]:
def plot_change_maps(im_list):
    """Compute and plot change maps"""

    # Run the algorithm with median filter and at 1% significance.
    result = ee.Dictionary(change_maps(im_list, median=True, alpha=0.01))

    # Extract the change maps and export to assets.
    cmap = ee.Image(result.get('cmap'))
    smap = ee.Image(result.get('smap'))
    fmap = ee.Image(result.get('fmap'))
    bmap = ee.Image(result.get('bmap'))
    cmaps = (
        ee.Image
        .cat(cmap, smap, fmap, bmap)
        .rename(['cmap', 'smap', 'fmap']+timestamplist[1:]))
    cmaps = cmaps.updateMask(cmaps.gt(0))
    location = aoi.centroid().coordinates().getInfo()[::-1]

    # create parameters for cmap
    palette = ['black', 'cyan']
    params = {'min': 0, 'max': 1, 'palette': palette}

    # create map with layers

    Map = geemap.Map(location=location, zoom_start=15)

    # Different basemaps. you can select or deselect on image itself
    Map.add_basemap('SATELLITE')
    Map.add_basemap('USGS NAIP Imagery NDVI')
    Map.add_basemap('USGS NAIP Imagery False Color')

    # Our Cmaps layer
    Map.addLayer(cmaps.select(slide_image), params, 'slide_image')

    return Map

In [61]:
plot_change_maps(im_list)

NameError: name 'change_maps' is not defined

## Landslide 8728

In [ ]:
# # Sentinel-1 images availability of landslide 8728
# event_date = ee.Date('2015-10-11')

# # Select one year surrounding the event
# start_date = event_date.advance(-180, 'days')
# end_date = event_date.advance(180, 'days')

# 'Sentinel-1 image data range is between {} and {}'.format(
#     start_date.format('YYYY-MM-dd').getInfo(),
#     end_date.format('YYYY-MM-dd').getInfo())

In [ ]:
# # define the center coordinates of region of interest
# center_point = ee.Geometry.Point([-104.832, 38.7612])

# # Create an Earth Engine Bbox using the center coordinates and dimensions
# width = 1000
# #height_degrees = 600

# bbox = center_point.buffer(width / 2).bounds()

# # Print the bounding box coordinates with four decimal places
# print("Bounding box coordinates: ", [[round(x, 4) for x in coords] for coords
#                                      in bbox.coordinates().getInfo()[0]])

# bbox

In [ ]:
# # Collect and filter Sentinel-1 images by time and region of interest
# start_date = '2015-04-14'
# end_date = '2016-04-08'
# sentinel_1 = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
#               .filterBounds(bbox)
#               .filterDate(start_date, end_date)
#               .filter(ee.Filter.listContains('transmitterReceiverPolarisation'
#                                              ,'VV'))
#               .filter(ee.Filter.listContains('transmitterReceiverPolarisation'
#                                              ,'VH'))
#              )

# image_collection = sentinel_1.filter(ee.Filter.eq('orbitProperties_pass', 
#                                                   'ASCENDING'))

# orbit_num = (image_collection.aggregate_array('relativeOrbitNumber_start')
#              .getInfo())
# if orbit_num:
#     orbit_num = orbit_num[0]

# look_angle = (image_collection.aggregate_array('orbitProperties_pass')
#               .getInfo())
# if look_angle and len(look_angle) > 0:
#     look_angle = look_angle[0]

# print('The Relative Orbit Number for ROI is: ', orbit_num)
# print('The orbitology is: ', look_angle)

# image_collection